<a href="https://colab.research.google.com/github/PhanDung520/wake_word/blob/main/automatic_model_training_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training your own openWakeWord models


**Quick-start:** If you just want to train a basic custom model for openWakeWord!

Follow the instructions for Step 1 below. Each time you change the wake word, click the play icon to the left of the title to generate a sample and make sure it sounds correct. The first time it takes a few minutes but subsequent runs will be quick.

Once you're satisfied with the pronounciation, go to the "Runtime" dropdown menu in the upper left of the page, and select "run all". Keep the tab open but feel free to do something else. After ~1 hour, your custom model will be ready and will automatically be downloaded to your computer!

If you are a Home Assistant user with the openWakeWord add-on, follow the instructions [here](https://github.com/home-assistant/addons/blob/master/openwakeword/DOCS.md#custom-wake-word-models) to install and enable your custom model.

---

If you are interested in learning more about the custom model training process (and increasing the accuracy of your custom models), read through each step in this notebook and try experimenting with different training parameters. If you have any questions or problems, feel free to start a discussion at the openWakeWord [repo](https://github.com/dscripka/openWakeWord/discussions).

In [1]:
# @title  { display-mode: "form" }
# @markdown # 1. Test Example Training Clip Generation
# @markdown Since openWakeWord models are trained on synthetic examples of your
# @markdown target wake word, it's a good idea to make sure that the examples
# @markdown sound correct. Type in your target wake word below, and run the
# @markdown cell to listen to it.
# @markdown
# @markdown Here are some tips that can help get the wake word to sound right:

# @markdown - If your wake word isn't being pronounced in the way
# @markdown you want, try spelling out the sounds phonetically with underscores
# @markdown separating each part.
# @markdown For example: "hey siri" --> "hey_seer_e".

# @markdown - Spell out numbers ("2" --> "two")

# @markdown - Avoid all punctuation except for "?" and "!", and remove unicode characters

import os
import sys
from IPython.display import Audio

# Ensure piper-sample-generator is in sys.path regardless of whether it was cloned this session
if "piper-sample-generator/" not in sys.path:
    sys.path.append("piper-sample-generator/")

if not os.path.exists("./piper-sample-generator"):
    !git clone https://github.com/rhasspy/piper-sample-generator
    !wget -O piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'
    !cd piper-sample-generator && git checkout 213d4d5

    # Install system dependencies
    !pip install piper-tts piper-phonemize-cross
    !pip install webrtcvad
    !pip install 'torch<=2.5' torchvision torchaudio

# Fix for generate_samples.py AttributeError when dataset is empty dict
generate_samples_path = "./piper-sample-generator/generate_samples.py"
if os.path.exists(generate_samples_path):
    with open(generate_samples_path, 'r') as f:
        lines = f.readlines()

    modified_lines = []
    found_line = False
    for line in lines:
        if 'sample_rate=dataset.features["audio"].sampling_rate,' in line:
            # Ensure proper indentation
            modified_line = line.replace(
                'sample_rate=dataset.features["audio"].sampling_rate,',
                '            sample_rate=(dataset.features["audio"].sampling_rate if isinstance(dataset, datasets.Dataset) else SAMPLING_RATE),'
            )
            modified_lines.append(modified_line)
            found_line = True
        else:
            modified_lines.append(line)

    if found_line:
        with open(generate_samples_path, 'w') as f:
            f.writelines(modified_lines)
        print(f"Successfully patched {generate_samples_path}")
    else:
        print(f"Warning: Could not find target line in {generate_samples_path} for modification.")
else:
    print(f"Error: {generate_samples_path} not found for patching.")

# Move import outside the if block to ensure it's always executed
from generate_samples import generate_samples

target_word = 'lan vee' # @param {type:"string"}

def text_to_speech(text):
    generate_samples(text = text,
                max_samples=1,
                length_scales=[1.1],
                noise_scales=[0.7], noise_scale_ws = [0.7],
                output_dir = './', batch_size=1, auto_reduce_batch_size=True,
                file_names=["test_generation.wav"]
                )

text_to_speech(target_word)
Audio("test_generation.wav", autoplay=True)


Cloning into 'piper-sample-generator'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 161 (delta 64), reused 62 (delta 50), pack-reused 69 (from 1)
Receiving objects: 100% (161/161), 1.04 MiB | 2.22 MiB/s, done.
Resolving deltas: 100% (74/74), done.
--2026-02-26 04:43:30--  https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/642029941/73f4af3c-7cf8-4547-a7b9-3bd29e7f3c33?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-26T05%3A38%3A37Z&rscd=attachment%3B+filename%3Den_US-libritts_r-medium.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-99

/content/piper-sample-generator/generate_samples.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch_model = torch.load(model_path)


In [4]:
# @title  { display-mode: "form" }
# @markdown # 2. Download Data
# @markdown Training custom models requires downloading a wide variety of data
# @markdown that will help make the model perform well in real-world scenarios.
# @markdown This example notebook will download small samples of background noise,
# @markdown music, and Room Impulse Responses (to add echo). This will still produce
# @markdown a custom model that performs well, but if you are interested in adding even more,
# @markdown feel free to extend this notebook to download the full datasets and even add
# @markdown your own!
# @markdown
# @markdown Downloading this example data will usually take about 15 minutes.

# @markdown **Important note!** The data downloaded here has a mixture of different
# @markdown licenses and usage restrictions. As such, any custom models trained with this
# @markdown data should be considered as appropriate for **non-commercial** personal use only.

# ## Install all dependencies
# !pip install datasets
# !pip install scipy
# !pip install tqdm

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# --- BEGIN FIX FOR NUMPY IMPORTERROR ---
# Aggressively uninstall conflicting packages and reinstall a known compatible numpy version
!pip uninstall numpy pandas datasets -y
!pip install numpy==1.23.4 # A stable version known to work with pandas/datasets pre-numpy 2.0
# --- END FIX FOR NUMPY IMPORTERROR ---

# install openwakeword (full installation to support training)
!git clone https://github.com/dscripka/openwakeword
!pip install -e ./openwakeword --no-deps
# !cd openwakeword

# install other dependencies
!pip install mutagen==1.47.0
!pip install torchinfo==1.8.0
!pip install torchmetrics==1.2.0
!pip install speechbrain==0.5.14
!pip install audiomentations==0.33.0
!pip install torch-audiomentations==0.11.0
!pip install acoustics==0.2.6
# !pip uninstall tensorflow -y
# !pip install tensorflow-cpu==2.8.1
# !pip install protobuf==3.20.3
# !pip install tensorflow_probability==0.16.0
# !pip install onnx_tf==1.10.0
!pip install onnxruntime==1.22.1 ai_edge_litert==1.4.0 onnxsim

# --- BEGIN FIX FOR ONNX DEPENDENCY CONFLICT ---
# Install onnx version required by onnx2tf first
!pip install onnx==1.20.1
!pip install onnx2tf
# --- END FIX FOR ONNX DEPENDENCY CONFLICT ---

# Original onnx install line (now modified):
# !pip install onnx==1.19.1

!pip install onnx_graphsurgeon
!pip install sng4onnx
!pip install pronouncing==0.2.0

# Removed redundant numpy uninstall, as it's handled at the start
# !pip uninstall numpy -y

# Reinstall datasets and pandas after numpy is fixed
!pip install pandas # Let it install a compatible version with numpy==1.23.4
!pip install datasets==2.14.6
!pip install deep-phonemizer==0.0.19

# Download required models (workaround for Colab)
import os
os.makedirs("./openwakeword/openwakeword/resources/models", exist_ok=True)
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx -O ./openwakeword/openwakeword/resources/models/embedding_model.onnx
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite -O ./openwakeword/openwakeword/resources/models/embedding_model.tflite
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx -O ./openwakeword/openwakeword/resources/models/melspectrogram.onnx
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite -O ./openwakeword/openwakeword/resources/models/melspectrogram.tflite

# Imports
import sys

if "piper-sample-generator/" not in sys.path:
    sys.path.append("piper-sample-generator/")
from generate_samples import generate_samples

import numpy as np
import torch
import sys
from pathlib import Path
import uuid
import yaml
import datasets
import scipy
from tqdm import tqdm

## Download all data

## Download MIR RIR data (takes about ~2 minutes)
output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    !git lfs install
    !git clone https://huggingface.co/datasets/davidscripka/MIT_environmental_impulse_responses
    rir_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path("./MIT_environmental_impulse_responses/16khz").glob("*.wav")]}).cast_column("audio", datasets.Audio())
    # Save clips to 16-bit PCM wav files
    for row in tqdm(rir_dataset):
        name = row['audio']['path'].split('/')[-1]
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

## Download noise and background audio (takes about ~3 minutes)

# Audioset Dataset (https://research.google.com/audioset/dataset/index.html)
# Download one part of the audioset .tar files, extract, and convert to 16khz
# For full-scale training, it's recommended to download the entire dataset from
# https://huggingface.co/datasets/agkphysics/AudioSet, and
# even potentially combine it with other background noise datasets (e.g., FSD50k, Freesound, etc.)

# --- COMMENTED OUT DUE TO 404 ERROR ---
# if not os.path.exists("audioset"):
#     os.mkdir("audioset")

#     fname = "bal_train09.tar"
#     out_dir = f"audioset/{fname}"
#     link = "https://huggingface.co/datasets/agkphysics/AudioSet/resolve/main/data/" + fname
#     !wget -O {out_dir} {link}
#     !cd audioset && tar -xvf bal_train09.tar

#     output_dir = "./audioset_16k"
#     if not os.path.exists(output_dir):
#         os.mkdir(output_dir)

#     # Save clips to 16-bit PCM wav files
#     audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path("audioset/audio").glob("**/*.flac")]})
#     audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
#     for row in tqdm(audioset_dataset):
#         name = row['audio']['path'].split('/')[-1].replace(".flac", ".wav")
#         scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))
# --- END COMMENTED OUT ---

# Free Music Archive dataset
# https://github.com/mdeff/fma

output_dir = "./fma"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
    fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

    # Save clips to 16-bit PCM wav files
    n_hours = 1  # use only 1 hour of clips for this example notebook, recommend increasing for full-scale training
    for i in tqdm(range(n_hours*3600//30)):  # this works because the FMA dataset is all 30 second clips
        row = next(fma_dataset)
        name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))
        i += 1
        if i == n_hours*3600//30:
            break

# Download pre-computed openWakeWord features for training and validation

# training set (~2,000 hours from the ACAV100M Dataset)
# See https://huggingface.co/datasets/davidscripka/openwakeword_features for more information
if not os.path.exists("./openwakeword_features_ACAV100M_2000_hrs_16bit.npy"):
    !wget https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy

# validation set for false positive rate estimation (~11 hours)
if not os.path.exists("validation_set_features.npy"):
    !wget https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/validation_set_features.npy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: datasets 2.14.6
Uninstalling datasets-2.14.6:
  Successfully uninstalled datasets-2.14.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 75.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
fatal: destination path 

  Using cached numpy-2.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openwakeword 0.6.0 requires speexdsp-ns<1,>=0.1.2; platform_system == "Linux", which is not installed.
acoustics 0.2.6 requires pandas>=0.15, which is not installed.
pandas-gbq 0.30.0 requires pandas>=1.1.4, which is not installed.
prophet 1.3.0 requires pandas>=1.0.4, which is not installed.
dask-cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, which is not installed.
mizani 0.13.5 requires pandas>=2.2.0, which is not installed.
segregation 2.5.3 requires pandas, which is 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 61.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
dask-cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, but you have pandas 3.0.1 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.1 which is incompatible.
cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, but you have pandas 3.0.1 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.1 which is incompatible.
db-dtypes 1.5.0 requires pandas<3.0.0,>=1.5.3, but you have pandas 3.0.1 which is inco

  Using cached onnxruntime-1.22.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.9 kB)
  Using cached ai_edge_litert-1.4.0-cp312-cp312-manylinux_2_17_x86_64.whl.metadata (1.9 kB)
Using cached onnxruntime-1.22.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.5 MB)
Using cached ai_edge_litert-1.4.0-cp312-cp312-manylinux_2_17_x86_64.whl (11.2 MB)
  Attempting uninstall: ai_edge_litert
    Found existing installation: ai-edge-litert 2.1.2
    Uninstalling ai-edge-litert-2.1.2:
      Successfully uninstalled ai-edge-litert-2.1.2
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.24.1
    Uninstalling onnxruntime-1.24.1:
      Successfully uninstalled onnxruntime-1.24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openwakeword 0.6.0 requires speexdsp-ns<1,>=0.1.2; platform_system == "Linux",

  Using cached datasets-2.14.6-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.14.6-py3-none-any.whl (493 kB)


--2026-02-26 05:13:19--  https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/497407399/0233db07-b8db-4fc3-b026-b75d77fd7ae6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-26T06%3A13%3A01Z&rscd=attachment%3B+filename%3Dembedding_model.onnx&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-02-26T05%3A12%3A31Z&ske=2026-02-26T06%3A13%3A01Z&sks=b&skv=2018-11-09&sig=Fjm7caFoQDvWPi8ir%2FK0rSgQ3kAnutcFMZjrbnnHCuw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc3MjA4MzA5OSwibmJmIjoxNzcyMDgyNzk5LCJwYXRoIjoicmVsZWFzZWFzc2V0

In [5]:
# @title  { display-mode: "form" }
# @markdown # 3. Train the Model
# @markdown Now that you have verified your target wake word and downloaded the data,
# @markdown the last step is to adjust the training paramaters (or keep
# @markdown the defaults below) and start the training!

# @markdown Each paramater controls a different aspect of training:
# @markdown - `number_of_examples` controls how many examples of your wakeword
# @markdown are generated. The default (1,000) usually produces a good model,
# @markdown but between 30,000 and 50,000 is often the best.

# @markdown - `number_of_training_steps` controls how long to train the model.
# @markdown Similar to the number of examples, the default (10,000) usually works well
# @markdown but training longer usually helps.

# @markdown - `false_activation_penalty` controls how strongly false activations
# @markdown are penalized during the training process. Higher values can make the model
# @markdown much less likely to activate when it shouldn't, but may also cause it
# @markdown to not activate when the wake word isn't spoken clearly and there is
# @markdown background noise.

# @markdown With the default values shown below,
# @markdown this takes about 30 - 60 minutes total on the normal CPU Colab runtime.
# @markdown If you want to train on more examples or train for longer,
# @markdown try changing the runtime type to a GPU to significantly speedup
# @markdown the example generating and model training.

# @markdown When the model finishes training, you can navigate to the `my_custom_model` folder
# @markdown in the file browser on the left (click on the folder icon), and download
# @markdown the [your target wake word].onnx or  <your target wake word>.tflite files.
# @markdown You can then use these as you would any other openWakeWord model!

# Load default YAML config file for training
import yaml
config = yaml.load(open("openwakeword/examples/custom_model.yml", 'r').read(), yaml.Loader)

# Modify values in the config and save a new version
number_of_examples = 30000 # @param {type:"slider", min:100, max:50000, step:50}
number_of_training_steps = 10000  # @param {type:"slider", min:0, max:50000, step:100}
false_activation_penalty = 1500  # @param {type:"slider", min:100, max:5000, step:50}
config["target_phrase"] = [target_word]
config["model_name"] = config["target_phrase"][0].replace(" ", "_")
config["n_samples"] = number_of_examples
config["n_samples_val"] = max(500, number_of_examples//10)
config["steps"] = number_of_training_steps
config["target_accuracy"] = 0.5
config["target_recall"] = 0.25
config["output_dir"] = "./my_custom_model"
config["max_negative_weight"] = false_activation_penalty

config["background_paths"] = ['./audioset_16k', './fma']  # multiple background datasets are supported
config["false_positive_validation_data_path"] = "validation_set_features.npy"
config["feature_data_files"] = {"ACAV100M_sample": "openwakeword_features_ACAV100M_2000_hrs_16bit.npy"}

with open('my_model.yaml', 'w') as file:
    documents = yaml.dump(config, file)

# Generate clips
!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --generate_clips

# Step 2: Augment the generated clips

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --augment_clips

# Step 3: Train model

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --train_model

# # Manually save to tflite as this doesn't work right in colab (broken in python 3.11, default in Colab as of January 2025)
# def convert_onnx_to_tflite(onnx_model_path, output_path):
#     """Converts an ONNX version of an openwakeword model to the Tensorflow tflite format."""
#     # imports
#     import onnx
#     import logging
#     import tempfile
#     from onnx_tf.backend import prepare
#     import tensorflow as tf

#     # Convert to tflite from onnx model
#     onnx_model = onnx.load(onnx_model_path)
#     tf_rep = prepare(onnx_model, device="CPU")
#     with tempfile.TemporaryDirectory() as tmp_dir:
#         tf_rep.export_graph(os.path.join(tmp_dir, "tf_model"))
#         converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(tmp_dir, "tf_model"))
#         tflite_model = converter.convert()

#         logging.info(f"####\nSaving tflite mode to '{output_path}'")
#         with open(output_path, 'wb') as f:
#             f.write(tflite_model)

#     return None

# convert_onnx_to_tflite(f"my_custom_model/{config['model_name']}.onnx", f"my_custom_model/{config['model_name']}.tflite")

# Convert ONNX model to tflite using `onnx2tf` library (works for python 3.11 as of January 2025)
onnx_model_path = f"my_custom_model/{config['model_name']}.onnx"
name1, name2 = f"my_custom_model/{config['model_name']}_float32.tflite", f"my_custom_model/{config['model_name']}.tflite"
!onnx2tf -i {onnx_model_path} -o my_custom_model/ -kat onnx____Flatten_0
!mv {name1} {name2}

# Automatically download the trained model files
from google.colab import files

files.download(f"my_custom_model/{config['model_name']}.onnx")
files.download(f"my_custom_model/{config['model_name']}.tflite")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
DEBUG:generate_samples:Batch 83/4285 complete
DEBUG:generate_samples:Batch 84/4285 complete
DEBUG:generate_samples:Batch 85/4285 complete
DEBUG:generate_samples:Batch 86/4285 complete
DEBUG:generate_samples:Batch 87/4285 complete
DEBUG:generate_samples:Batch 88/4285 complete
DEBUG:generate_samples:Batch 89/4285 complete
DEBUG:generate_samples:Batch 90/4285 complete
DEBUG:generate_samples:Batch 91/4285 complete
DEBUG:generate_samples:Batch 92/4285 complete
DEBUG:generate_samples:Batch 93/4285 complete
DEBUG:generate_samples:Batch 94/4285 complete
DEBUG:generate_samples:Batch 95/4285 complete
DEBUG:generate_samples:Batch 96/4285 complete
DEBUG:generate_samples:Batch 97/4285 complete
DEBUG:generate_samples:Batch 98/4285 complete
DEBUG:generate_samples:Batch 99/4285 complete
DEBUG:generate_samples:Batch 100/4285 complete
DEBUG:generate_samples:Batch 101/4285 complete
DEBUG:generate_samples:Batch 102/4285 complete
DEBUG:generate_sampl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>